# AWS Resource Cleanup 🧹

**Safely clean up AWS resources to avoid ongoing charges**

[![AWS](https://img.shields.io/badge/AWS-Ready-orange.svg)](https://aws.amazon.com/)

---

## ⚠️ Safety Features
- ✅ **Dry-run by default** - Preview before deleting
- ✅ **Interactive confirmation** - Must confirm deletions  
- ✅ **Detailed logging** - See what's happening

## 🎯 What Gets Cleaned
1. **SageMaker**: Endpoints, configs, models, training jobs
2. **S3**: Buckets with SageMaker artifacts
3. **CloudWatch**: Alarms

## 💰 Why This Matters
- Endpoints: $0.50-2.00/hour (~$360-1440/month)
- S3: $0.023/GB/month
- CloudWatch: $0.30/metric/month

**Avoid surprise bills!** 💸


In [4]:
# Install boto3 if needed
try:
    import boto3
    print("✅ boto3 available")
except:
    !pip install -q boto3
    import boto3
    print("✅ boto3 installed")

from datetime import datetime
print(f"\n✅ Setup complete!")
print(f"   boto3 version: {boto3.__version__}")


## ⚙️ Configuration


In [5]:
# Configuration
AWS_REGION = 'us-east-1'  # Change to your region
DRY_RUN = True  # Set to False to actually delete

print("="*60)
print("CLEANUP CONFIGURATION")
print("="*60)
print(f"Region: {AWS_REGION}")
print(f"Mode: {'DRY RUN (preview)' if DRY_RUN else '⚠️  DELETION MODE'}")
print("="*60)

if not DRY_RUN:
    print("\n⚠️  WARNING: Deletion mode ACTIVE!")


In [6]:
# ⚙️ AWS Credentials Setup
# Run this cell first to configure AWS credentials

import os

print("="*60)
print("AWS CREDENTIALS CHECK")
print("="*60)

# Check if credentials are already configured
try:
    # Try to get caller identity
    test_client = boto3.client('sts', region_name=AWS_REGION)
    identity = test_client.get_caller_identity()
    print("✅ AWS credentials are configured!")
    print(f"   Account: {identity['Account']}")
    print(f"   User: {identity['Arn'].split('/')[-1]}")
    print("="*60)
    credentials_configured = True
except Exception as e:
    print("❌ AWS credentials NOT configured")
    print("="*60)
    credentials_configured = False
    
    print("\n📋 How to configure AWS credentials:\n")
    print("Option 1: Using AWS CLI (Recommended)")
    print("-" * 60)
    print("1. Install AWS CLI: https://aws.amazon.com/cli/")
    print("2. Run: aws configure")
    print("3. Enter your Access Key ID and Secret Access Key")
    print("   (Get these from AWS Console → IAM → Users → Security credentials)\n")
    
    print("Option 2: Set Environment Variables")
    print("-" * 60)
    print("In a terminal, run:")
    print("  export AWS_ACCESS_KEY_ID='your_access_key'")
    print("  export AWS_SECRET_ACCESS_KEY='your_secret_key'")
    print("  export AWS_DEFAULT_REGION='us-east-1'\n")
    
    print("Option 3: Configure in this notebook (Quick setup)")
    print("-" * 60)
    print("Uncomment and fill in the code below:\n")
    print("""
# import os
# os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key_here'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_access_key_here'
# os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
# print("✅ Credentials set in notebook")
    """)
    
    print("\n⚠️  After configuring credentials, restart the kernel and run cells again.\n")
    print("="*60)


In [7]:
# Initialize AWS clients
try:
    sagemaker = boto3.client('sagemaker', region_name=AWS_REGION)
    s3 = boto3.client('s3', region_name=AWS_REGION)
    s3_resource = boto3.resource('s3', region_name=AWS_REGION)
    cloudwatch = boto3.client('cloudwatch', region_name=AWS_REGION)
    
    # Test connection
    identity = boto3.client('sts', region_name=AWS_REGION).get_caller_identity()
    print(f"✅ Connected to AWS!")
    print(f"   Account: {identity['Account']}")
    print(f"   User: {identity['Arn'].split('/')[-1]}")
    
except Exception as e:
    print("❌ Failed to connect to AWS")
    print(f"   Error: {str(e)}\n")
    print("⚠️  Please configure AWS credentials using the cell above")
    print("   Then restart kernel and run cells in order")
    raise


## 🔍 Check Resources

Run cells below to see what resources exist


In [ ]:
# Check SageMaker Endpoints (Most expensive!)
print("\n" + "="*60)
print("SAGEMAKER ENDPOINTS")
print("="*60)

endpoints = sagemaker.list_endpoints()['Endpoints']
if endpoints:
    print(f"Found {len(endpoints)} endpoint(s):\n")
    for ep in endpoints:
        print(f"  📍 {ep['EndpointName']}")
        print(f"     Status: {ep['EndpointStatus']}")
        print(f"     Created: {ep['CreationTime'].strftime('%Y-%m-%d')}\n")
else:
    print("✅ No endpoints found")


In [ ]:
# Check Endpoint Configs
print("\n" + "="*60)
print("ENDPOINT CONFIGURATIONS")
print("="*60)

configs = sagemaker.list_endpoint_configs()['EndpointConfigs']
if configs:
    print(f"Found {len(configs)} config(s):\n")
    for cfg in configs:
        print(f"  ⚙️  {cfg['EndpointConfigName']}\n")
else:
    print("✅ No configs found")


In [ ]:
# Check SageMaker Models
print("\n" + "="*60)
print("SAGEMAKER MODELS")
print("="*60)

models = sagemaker.list_models()['Models']
if models:
    print(f"Found {len(models)} model(s):\n")
    for model in models:
        print(f"  🤖 {model['ModelName']}\n")
else:
    print("✅ No models found")


In [ ]:
# Check S3 Buckets (SageMaker related)
print("\n" + "="*60)
print("S3 BUCKETS (SageMaker)")
print("="*60)

all_buckets = s3.list_buckets()['Buckets']
buckets = [b for b in all_buckets if 'sagemaker' in b['Name'].lower()]

if buckets:
    print(f"Found {len(buckets)} SageMaker bucket(s):\n")
    for bucket in buckets:
        print(f"  🪣 {bucket['Name']}\n")
else:
    print("✅ No SageMaker buckets found")


In [ ]:
# Check CloudWatch Alarms
print("\n" + "="*60)
print("CLOUDWATCH ALARMS")
print("="*60)

alarms = cloudwatch.describe_alarms()['MetricAlarms']
if alarms:
    print(f"Found {len(alarms)} alarm(s):\n")
    for alarm in alarms:
        print(f"  🔔 {alarm['AlarmName']}")
        print(f"     State: {alarm['StateValue']}\n")
else:
    print("✅ No alarms found")


## 🗑️ Cleanup

**To enable deletion:** Set `DRY_RUN = False` in configuration cell above


In [ ]:
# Run cleanup
if not DRY_RUN:
    print("⚠️  DELETION MODE ACTIVE\n")
    
    # Delete endpoints
    if endpoints:
        for ep in endpoints:
            sagemaker.delete_endpoint(EndpointName=ep['EndpointName'])
            print(f"✅ Deleted endpoint: {ep['EndpointName']}")
    
    # Delete configs
    if configs:
        for cfg in configs:
            sagemaker.delete_endpoint_config(EndpointConfigName=cfg['EndpointConfigName'])
            print(f"✅ Deleted config: {cfg['EndpointConfigName']}")
    
    # Delete models
    if models:
        for model in models:
            sagemaker.delete_model(ModelName=model['ModelName'])
            print(f"✅ Deleted model: {model['ModelName']}")
    
    # Delete CloudWatch alarms
    if alarms:
        alarm_names = [a['AlarmName'] for a in alarms]
        cloudwatch.delete_alarms(AlarmNames=alarm_names)
        print(f"✅ Deleted {len(alarm_names)} alarm(s)")
    
    print("\n✅ CLEANUP COMPLETE!")
    
else:
    print("💡 DRY RUN mode - No changes made")
    print("   Set DRY_RUN=False to actually delete resources")


---

## ✅ Done!

**Always verify deletions in AWS Console:**
- 📊 [SageMaker](https://console.aws.amazon.com/sagemaker/)
- 🪣 [S3](https://console.aws.amazon.com/s3/)
- 📈 [CloudWatch](https://console.aws.amazon.com/cloudwatch/)

**💰 Cost Savings:**
- Stopping 1 endpoint: ~$360-1440/month
- Deleting old S3 data: ~$0.023/GB/month
- Removing unused alarms: ~$0.30/metric/month

**🔒 Security:** This notebook never deletes S3 buckets automatically for safety.
